In [4]:
import face_recognition
import cv2

In [5]:
webcam_video_stream = cv2.VideoCapture(0)
all_face_locations = []
AGE_GENDER_MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

gender_label_list = ['Male', 'Female']
gender_protext = 'dataset/gender_deploy.prototxt'
gender_caffemodel = 'dataset/gender_net.caffemodel'
gender_cov_net = cv2.dnn.readNet(gender_caffemodel, gender_protext)

age_label_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
age_protext = 'dataset/age_deploy.prototxt'
age_caffemodel = 'dataset/age_net.caffemodel'
age_cov_net = cv2.dnn.readNet(age_caffemodel, age_protext)

In [6]:
while True:
    ret, current_frame = webcam_video_stream.read()
    current_frame_small = cv2.resize(current_frame, (0,0), fx=0.25, fy=0.25)
    all_face_locations = face_recognition.face_locations(current_frame_small, number_of_times_to_upsample=2, model='hog')

    for index, current_face_location in enumerate(all_face_locations):

        # splitting the tuple to get the four position values of current face
        top_pos, right_pos, bottom_pos, left_pos = current_face_location
        top_pos *= 4
        right_pos *= 4
        bottom_pos *= 4
        left_pos *= 4

        # slicing the current face from main image
        current_face_image = current_frame[top_pos:bottom_pos, left_pos:right_pos]

        # create blob of current face slice
        current_face_image_blob = cv2.dnn.blobFromImage(current_face_image, 1, (227, 227), AGE_GENDER_MODEL_MEAN_VALUES, swapRB = False)

        # predict gender
        gender_cov_net.setInput(current_face_image_blob)
        gender_predictions = gender_cov_net.forward()
        gender = gender_label_list[gender_predictions[0].argmax()]

        # predict age
        age_cov_net.setInput(current_face_image_blob)
        age_predictions = age_cov_net.forward()
        age = age_label_list[age_predictions[0].argmax()]

        # draw rectangle around the face detected
        cv2.rectangle(current_frame, (left_pos, top_pos), (right_pos, bottom_pos), (0, 0, 255), 2)

        # Draw a label with a text below the face
        cv2.rectangle(current_frame, (left_pos, bottom_pos - 35), (right_pos, bottom_pos), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(current_frame, gender+" "+age+" "+"yrs", (left_pos+6, bottom_pos-6), font, 0.5, (255,255,255), 1)


    cv2.imshow('Webcam Video', current_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


webcam_video_stream.release()
cv2.destroyAllWindows()